In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
train_dataset = datasets.ImageFolder('./tiny-imagenet-200/train', transform=train_transforms)
train_dir = './tiny-imagenet-200/train'
classes = os.listdir(train_dir)
print("Folders found:", classes)
print("Number of classes:", len(classes))

Folders found: ['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815834', 'n02823428', 'n02837789', 'n0284131

In [4]:
val_dir = './tiny-imagenet-200/val'
annotation_file = os.path.join(val_dir, 'val_annotations.txt')
new_val_dir = './tiny-imagenet-200/val-images-by-class'
val_dataset = datasets.ImageFolder(new_val_dir, transform=train_transforms)
classes = os.listdir('./tiny-imagenet-200/val-images-by-class')
print(classes)
print(len(classes))

['n01443537', 'n01629819', 'n01641577', 'n01644900', 'n01698640', 'n01742172', 'n01768244', 'n01770393', 'n01774384', 'n01774750', 'n01784675', 'n01855672', 'n01882714', 'n01910747', 'n01917289', 'n01944390', 'n01945685', 'n01950731', 'n01983481', 'n01984695', 'n02002724', 'n02056570', 'n02058221', 'n02074367', 'n02085620', 'n02094433', 'n02099601', 'n02099712', 'n02106662', 'n02113799', 'n02123045', 'n02123394', 'n02124075', 'n02125311', 'n02129165', 'n02132136', 'n02165456', 'n02190166', 'n02206856', 'n02226429', 'n02231487', 'n02233338', 'n02236044', 'n02268443', 'n02279972', 'n02281406', 'n02321529', 'n02364673', 'n02395406', 'n02403003', 'n02410509', 'n02415577', 'n02423022', 'n02437312', 'n02480495', 'n02481823', 'n02486410', 'n02504458', 'n02509815', 'n02666196', 'n02669723', 'n02699494', 'n02730930', 'n02769748', 'n02788148', 'n02791270', 'n02793495', 'n02795169', 'n02802426', 'n02808440', 'n02814533', 'n02814860', 'n02815834', 'n02823428', 'n02837789', 'n02841315', 'n02843684'

In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [6]:
num_classes = 200           # Tiny ImageNet has 200 classes
learning_rate = 0.01
model = models.vgg16(weights='IMAGENET1K_V1')

# Modify the final fully connected layer to match the number of classes in Tiny ImageNet
for idx, layer in enumerate(model.features):
    if idx < 10:  # adjust this threshold based on your experiments
        for param in layer.parameters():
            param.requires_grad = False
    else:
        for param in layer.parameters():
            param.requires_grad = True

model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.7),  # Increased dropout probability
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.7),  # Increased dropout probability
    nn.Linear(4096, 200)  # Adjust to the number of classes in Tiny ImageNet
)

# model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [7]:
train_losses = []
val_losses = []
train_accs = []
val_accs = []
best_val_loss = float('inf')
patience = 5
trigger_times = 0
num_epochs = 50

for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    
    # Reset loss accumulators for this epoch
    running_train_loss = 0.0
    running_train_corrects = 0

    # Training Phase
    model.train()
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_train_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        running_train_corrects += torch.sum(preds == labels.data)

    scheduler.step()
    
    # Compute average training loss for the epoch
    epoch_train_loss = running_train_loss / len(train_loader.dataset)
    epoch_train_acc = running_train_corrects.float() / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)
    train_accs.append(epoch_train_acc.item())

    # Validation Phase
    model.eval()
    running_val_loss = 0.0
    running_val_corrects = 0

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc="Validation"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            running_val_corrects += torch.sum(preds == labels.data)
    
    epoch_val_loss = running_val_loss / len(val_loader.dataset)
    epoch_val_acc = running_val_corrects.float() / len(val_loader.dataset)
    val_losses.append(epoch_val_loss)
    val_accs.append(epoch_val_acc.item())
    
    print(f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f}")
    print(f"Val Loss: {epoch_val_loss:.4f}   | Val Acc: {epoch_val_acc:.4f}")
    print('-' * 30)

    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trigger_times += 1
        if trigger_times > patience:
            print("Early stopping")
            break

Epoch 1/50


Validation: 100%|██████████| 157/157 [00:11<00:00, 14.06it/s]


Train Loss: 4.4294 | Train Acc: 0.0831
Val Loss: 3.5779   | Val Acc: 0.1976
------------------------------
Epoch 2/50


Validation: 100%|██████████| 157/157 [00:11<00:00, 13.84it/s]


Train Loss: 3.5582 | Train Acc: 0.2067
Val Loss: 3.2091   | Val Acc: 0.2701
------------------------------
Epoch 3/50


...

Train Loss: 1.9381 | Train Acc: 0.5330
Val Loss: 2.1714   | Val Acc: 0.4945
------------------------------
Epoch 27/50


Validation: 100%|██████████| 157/157 [00:11<00:00, 14.10it/s]


Train Loss: 1.9329 | Train Acc: 0.5351
Val Loss: 2.1568   | Val Acc: 0.4954
------------------------------
Epoch 28/50


Validation: 100%|██████████| 157/157 [00:11<00:00, 14.06it/s]

Train Loss: 1.9277 | Train Acc: 0.5355
Val Loss: 2.1728   | Val Acc: 0.4942
------------------------------
Early stopping
